This is a Feed forward Neural Network

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn, rnn_cell

input > weights > hidden layer 1 (activation) > weights > hidden layer 2 (activation) > weights > output layer

Compare output to intended output using cost or loss function(cross entropy)                                               
optimization function (optimizer) > minimize cost (AdamOptimizer)

backpropagation (going backwards and manupulating the weights)

feed forward + backpropagation = epoch

In [2]:
mnist = input_data.read_data_sets("tmp/data/", one_hot = True)

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
hm_epochs = 10
n_classes = 10
batch_size = 128

chunk_size = 28
n_chunks = 28
rnn_size = 256

In [4]:
x = tf.placeholder('float', [None, n_chunks, chunk_size])
y = tf.placeholder('float')
print ("x = ", x)
print ("Y = ", y)

('x = ', <tf.Tensor 'Placeholder:0' shape=(?, 28, 28) dtype=float32>)
('Y = ', <tf.Tensor 'Placeholder_1:0' shape=<unknown> dtype=float32>)


In [5]:
def neural_network_model(x):
    
    layer = {'weights': tf.Variable(tf.random_normal([rnn_size, n_classes])),
                      'biases': tf.Variable(tf.random_normal([n_classes]))}
    
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunks, 0)
    
    lstm_cell = rnn_cell.BasicLSTMCell(rnn_size)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    output = tf.add(tf.matmul(outputs[-1], layer['weights']), layer['biases'])
    
    return output
    

In [6]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )    
    optimizer = tf.train.AdamOptimizer().minimize(cost)
     
    
    with tf.Session() as sess: 
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))
                
                _, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                
                epoch_loss += c
            print('Epoch ', epoch, ' Completed out of ', hm_epochs, ' Loss: ', epoch_loss)
            
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy: ', accuracy.eval({x: mnist.test.images.reshape((-1, n_chunks, chunk_size)), y: mnist.test.labels}))

In [7]:
train_neural_network(x)

('Epoch ', 0, ' Completed out of ', 10, ' Loss: ', 164.69143953919411)
('Epoch ', 1, ' Completed out of ', 10, ' Loss: ', 50.226362638175488)
('Epoch ', 2, ' Completed out of ', 10, ' Loss: ', 34.496672049164772)
('Epoch ', 3, ' Completed out of ', 10, ' Loss: ', 25.423532750923187)
('Epoch ', 4, ' Completed out of ', 10, ' Loss: ', 21.246823365334421)
('Epoch ', 5, ' Completed out of ', 10, ' Loss: ', 19.522982566035353)
('Epoch ', 6, ' Completed out of ', 10, ' Loss: ', 15.132899320800789)
('Epoch ', 7, ' Completed out of ', 10, ' Loss: ', 14.222527517005801)
('Epoch ', 8, ' Completed out of ', 10, ' Loss: ', 11.735110368521418)
('Epoch ', 9, ' Completed out of ', 10, ' Loss: ', 11.342846922110766)
('Accuracy: ', 0.9825002)
